In [1]:
import json
import numpy as np
import pandas as pd
from copy import copy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from constants import RUSSIAN_TIME_ZONES, rp5_columns
from interpolation import interpolate_mmx, create_patterns
from converters import convert_rp5_to_mmx, convert_raw_to_mmx, convert_mmx_to_mmcc_forecast, convert_raw_to_mmx
from loaders import load_rp5_stations, load_mm94_stations
from geographical import find_nearest_wmo_station, add_solar_angles

## Convert RP5 to Mmx

* Choose one mm94 station and finding the closeat wmo station

In [2]:
## find nearest wmo station
mm94_station_id = 113

wmo_station_id = find_nearest_wmo_station(mm94_station_id)

* Load data from the nearest wmo station

In [3]:
rp5 = load_rp5_stations(wmo_station_id)

* Convert from RP5 to Mmx format

In [5]:
mmx_forecast = convert_rp5_to_mmx(rp5)
mmx_forecast_interpolated = interpolate_mmx(mmx_forecast, 20)
#mmx_forecast_interpolated

* Convert from Mmx to MmccForecast format

In [5]:
mmcc_forecast = convert_mmx_to_mmcc_forecast(mmx_forecast_interpolated)

## Convert Raw to Mmx

* Load Raw mm94 data

In [19]:
%%time
raw = load_mm94_stations(mm94_station_id)
raw.head()

CPU times: user 945 ms, sys: 71.7 ms, total: 1.02 s
Wall time: 1.08 s


* Convert Raw to Mmx

In [20]:
mmx_rwis = convert_raw_to_mmx(raw)
mmx_rwis_interpolated = interpolate_mmx(mmx_rwis, 20)
mmx_rwis_interpolated = create_patterns(mmx_rwis_interpolated)

In [9]:
mmcc_rwis = convert_mmx_to_mmcc_forecast(mmx_rwis_interpolated)
mmcc_rwis

Column visibility can not be calculated
Column p_weather can not be calculated


,date_time_metro,wind_direction,wind_speed,precipitation_type,station_id,t_air,humidity,pressure,t_dew_point,precipitation_intensity,visibility,date_time_utc,p_weather,cloudiness
19,2008-03-23 21:00 UTC,9999.000000,9999.0,9999.0,1821,-2.733333,97.975000,9999.000000,-2.925000,9999.0,9999,2008-03-23 21:00:00,9999,9999.0
20,2008-03-23 21:20 UTC,9999.000000,9999.0,9999.0,1821,-2.666667,98.050000,9999.000000,-2.850000,9999.0,9999,2008-03-23 21:20:00,9999,9999.0
21,2008-03-23 21:40 UTC,9999.000000,9999.0,9999.0,1821,-2.600000,98.125000,9999.000000,-2.775000,9999.0,9999,2008-03-23 21:40:00,9999,9999.0
22,2008-03-23 22:00 UTC,9999.000000,9999.0,9999.0,1821,-2.533333,98.200000,9999.000000,-2.700000,9999.0,9999,2008-03-23 22:00:00,9999,9999.0
23,2008-03-23 22:20 UTC,9999.000000,9999.0,9999.0,1821,-2.466667,98.275000,9999.000000,-2.625000,9999.0,9999,2008-03-23 22:20:00,9999,9999.0
24,2008-03-23 22:40 UTC,9999.000000,9999.0,9999.0,1821,-2.400000,98.350000,9999.000000,-2.550000,9999.0,9999,2008-03-23 22:40:00,9999,9999.0
25,2008-03-23 23:00 UTC,9999.000000,9999.0,9999.0,1821,-2.333333,98.425000,9999.000000,-2.475000,9999.0,9999,2008-03-23 23:00:00,9999,9999.0
26,2008-03-23 23:20 UTC,9999.000000,9999.0,9999.0,1821,-2.266667,98.500000,9999.000000,-2.400000,9999.0,9999,2008-03-23 23:20:00,9999,9999.0
27,2008-03-23 23:40 UTC,9999.000000,9999.0,9999.0,1821,-2.200000,98.575000,9999.000000,-2.325000,9999.0,9999,2008-03-23 23:40:00,9999,9999.0
28,2008-03-24 00:40 UTC,9999.000000,9999.0,9999.0,1821,-2.050000,98.575000,9999.000000,-2.175000,9999.0,9999,2008-03-24 00:40:00,9999,9999.0
